# Imports, Data Downloads and Paths

In [ ]:
from fastai.conv_learner import *
from fastai.plots import *
from fastai.other.planet import f2

In [ ]:
PATH = '/home/msnow/data/planet/'
!ls {PATH}

# Multi-label models for Planet dataset

In [ ]:
metrics=[f2]
f_model = resnet34

In [ ]:
label_csv = f'{PATH}train_v2.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

In [ ]:
def get_data(sz, bs):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train-jpg', label_csv, bs, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test-jpg')

## Set the sz of the images and the batch size

In [ ]:
sz = 64
bs = 64
data = get_data(sz, bs)

Reduce the images to match the size you want, so that the nn doesn't have to do it on the fly

In [ ]:
datasz = data.resize(int(sz*1.3), 'tmp')

## All layers besides for the input and final layers are frozen

In [ ]:
bs_range = [16, 32, 64, 128, 256]
epoch_ct = [1, 2]
sz_range = [32, 64, 128, 256]
for bs in bs_range:
    for sz in sz_range:
        data = get_data(sz, bs)
        datasz = data.resize(int(sz*1.3), 'tmp')
        for epochs in epoch_ct:
            learn1 = ConvLearner.pretrained(f_model, datasz, metrics=metrics)
            lr = 0.2
            print(f'epochs: {epochs}, image size: {sz}, batch size: {bs}')
            %time learn1.fit(lr, 1, cycle_len=1)

In [ ]:
learn1 = ConvLearner.pretrained(f_model, datasz, metrics=metrics)
lr = 0.2
%time learn1.fit(lr, 1, cycle_len=1)

In [ ]:
learn1 = ConvLearner.pretrained(f_model, datasz, metrics=metrics)
lr = 0.2
%time learn1.fit(lr, 2, cycle_len=1)

## Unfreeze all the layers and use variable learning rates

In [ ]:
learn2 = ConvLearner.pretrained(f_model, datasz, metrics=metrics)
lrs = np.array([lr/9,lr/3,lr])
learn2.unfreeze()
%time learn2.fit(lrs, 1, cycle_len=1)

In [ ]:
learn2 = ConvLearner.pretrained(f_model, datasz, metrics=metrics)
lrs = np.array([lr/9,lr/3,lr])
learn2.unfreeze()
%time learn2.fit(lrs, 2, cycle_len=1)